In [1]:
%load_ext autoreload
%autoreload 2

In [66]:
from transformers import ElectraTokenizerFast
tokenizer = ElectraTokenizerFast.from_pretrained("monologg/koelectra-base-v3-discriminator")
tokenizer.tokenize("[CLS] 한국어 ELECTRA를 공유합니다. [SEP]")


['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]']

In [67]:
tokenizer.convert_tokens_to_ids(['[CLS]', '한국어', 'EL', '##EC', '##TRA', '##를', '공유', '##합니다', '.', '[SEP]'])

[2, 11229, 29173, 13352, 25541, 4110, 7824, 17788, 18, 3]

In [4]:
import os, sys

os.chdir('../')

In [5]:
!ls

__init__.py		  configurations	ipu_electra.ipynb
__pycache__		  exe_cache		ipu_train.py
bert_test.ipynb		  finetune		modeling_electra.py
checkpoints		  input.txt		pipeline_electra.py
configuration_electra.py  ipu_configuration.py	test.py


# KoELECTRA training

In [6]:
import yaml
from easydict import EasyDict

In [7]:
config_file = 'finetune/squad_configurations.yaml'

In [8]:
config = EasyDict(yaml.load(open(config_file).read(), Loader=yaml.Loader))

In [9]:
config

{'tokenizer_name': 'monologg/koelectra-base-v3-discriminator',
 'train_config': {'model_name_or_path': 'monologg/koelectra-base-v3-discriminator',
  'train_global_batch_size': 128,
  'train_micro_batch_size': 2,
  'train_replication_factor': 1,
  'train_device_iterations': 1,
  'num_epochs': 3,
  'sequence_length': 384,
  'train_layers_per_ipu': [2, 3, 4, 3],
  'train_recompute_checkpoint_every_layer': True,
  'train_embedding_serialization_factor': 1,
  'saved_model_name': 'checkpoints/squad_base_4x2'},
 'valid_config': {'valid_micro_batch_size': 4,
  'valid_replication_factor': 4,
  'valid_device_iterations': 2,
  'valid_layer_per_ipu': [11, 13],
  'valid_recompute_checkpoint_every_layer': False,
  'valid_embedding_serialization_factor': 1}}

In [10]:
config.train_config.saved_model_name

'checkpoints/squad_base_4x2'

In [11]:
config.valid_config.valid_recompute_checkpoint_every_layer

False

In [12]:
ipu_config = {
        "layers_per_ipu": config.train_config.train_layers_per_ipu,
        "recompute_checkpoint_every_layer":True,
        "embedding_serialization_factor": 1
    }

In [13]:
from datasets import load_dataset, load_metric
from pathlib import Path
from easydict import EasyDict 

import transformers
import popart
import poptorch
import torch

from pipeline_electra import PipelinedElectraForQuestionAnswering

from finetune.squad_preprocessing import prepare_train_features, \
                                prepare_validation_features, \
                                tokenizer, PadCollate, postprocess_qa_predictions

from finetune.run_squad_ipu import train, get_optimizer, ipu_options

# Load Datasets

In [14]:
datasets = load_dataset("squad_kor_v1", cache_dir=Path.home() / ".torch./dataset")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/60407 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5774 [00:00<?, ? examples/s]

Dataset squad_kor_v1 downloaded and prepared to /root/.torch./dataset/squad_kor_v1/squad_kor_v1/1.0.0/18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [19]:
datasets['train'][1]

{'id': '6566495-0-1',
 'title': '파우스트_서곡',
 'context': '1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.',
 'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?',
 'answers': {'text': ['1악장'], 'answer_start': [421]}}

In [37]:
train_dataset = datasets["train"].map(
    prepare_train_features,
    batched=True,
    num_proc=1,
    remove_columns=datasets["train"].column_names,
    load_from_cache_file=True,
)

# Create validation features from dataset
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    num_proc=1,
    remove_columns=datasets["validation"].column_names,
    load_from_cache_file=True,
)


# Electra-base configuration
config = transformers.ElectraConfig(embedding_size=768,
                                    hidden_size=768,
                                    intermediate_size = 1024*3,
                                    num_hidden_layers=12,
                                    num_attention_heads=12,
                                    hidden_dropout_prob=0.1,
                                    attention_probs_dropout_prob=0.1,
                                    layer_norm_eps=1e-12,
                                    vocab_size= 35000)
#gpu_model = transformers.ElectraForQuestionAnswering.from_pretrained("bhadresh-savani/electra-base-squad2", config=config)
ipu_config = {
    "layers_per_ipu": [2,3,4,3],
    "recompute_checkpoint_every_layer":True,
    "embedding_serialization_factor": 1
}

  0%|          | 0/88 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [38]:
ipu_config = EasyDict(ipu_config)
model = PipelinedElectraForQuestionAnswering.from_pretrained_transformers("monologg/koelectra-base-v3-discriminator", ipu_config, config=config)

model.parallelize().half().train()

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing PipelinedElectraForQuestionAnswering: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing PipelinedElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing PipelinedElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of PipelinedElectraForQuestionAnswering were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are new

--- Device Allocation ---
Embedding --> IPU 0
Encoder 0  --> IPU 0
Encoder 1  --> IPU 0
Encoder 2  --> IPU 1
Encoder 3  --> IPU 1
Encoder 4  --> IPU 1
Encoder 5  --> IPU 2
Encoder 6  --> IPU 2
Encoder 7  --> IPU 2
Encoder 8  --> IPU 2
Encoder 9  --> IPU 3
Encoder 10 --> IPU 3
Encoder 11 --> IPU 3
QA Outputs --> IPU 3


PipelinedElectraForQuestionAnswering(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): Layer

In [41]:
model_config = transformers.ElectraConfig.from_pretrained(config.train_config.model_name_or_path)

AttributeError: 'ElectraConfig' object has no attribute 'train_config'

In [ ]:
# Runtime Configuration
'''
set replication_factor to 1
it's mean the model using 8 IPU setted beforehand
'''
global_batch_size = 128
micro_batch_size = 2
replication_factor = 1
gradient_accumulation = int(global_batch_size / micro_batch_size / replication_factor)
device_iterations = 1
train_samples_per_iteration = global_batch_size * device_iterations
num_epochs = 3

train_opts = ipu_options(gradient_accumulation, replication_factor, device_iterations, train_option=True)

# Training
sequence_length = 384
train_dl = poptorch.DataLoader(train_opts,
                           train_dataset,
                           batch_size=micro_batch_size,
                           shuffle=True,
                           drop_last=False,
                           collate_fn=PadCollate(global_batch_size,
                                                 {"input_ids": 0,
                                                  "attention_mask": 0,
                                                  "token_type_ids": 0,
                                                  "start_positions": sequence_length,
                                                  "end_positions": sequence_length}))

optimizer = get_optimizer(model)
train(model, train_opts, optimizer, train_dl, num_epochs, train_samples_per_iteration)

model.save_pretrained("checkpoints/squad_base_4x2")


[05:51:21.311] [poptorch::python] [warning] The number of elements in the dataset (88524) is not divisible by the number of elements processed per step (128) and drop_last=False. The last tensor will have a batch size of 76. To avoid having to handle this special case switch to drop_last=True
Graph compilation:   3%|▎         | 3/100 [00:17<09:09]

# Inference

In [72]:
datasets['validation'][0]

{'id': '6548850-0-0',
 'title': '임종석',
 'context': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.',
 'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?',
 'answers': {'text': ['1989년 2월 15일'], 'answer_start': [0]}}

In [73]:
question = datasets['validation'][0]['question']
passage = datasets['validation'][0]['context']

In [74]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_encoding = tokenizer.encode_plus((question, passage))

# Extract inputs, add batch dimension
input_tensor = torch.tensor(input_encoding["input_ids"]).unsqueeze(0)
attention_tensor= torch.tensor(input_encoding["attention_mask"]).unsqueeze(0)
token_types=torch.tensor(input_encoding["token_type_ids"]).unsqueeze(0)
    
# Get model and load the fine-tuned weights
model = transformers.ElectraForQuestionAnswering.from_pretrained("checkpoints/squad_base_4x2")

In [75]:
# Solve task
outputs = model(input_tensor, attention_tensor, token_types)

# Extract answer
answer_start, answer_stop = outputs.start_logits.argmax(), outputs.end_logits.argmax()
answer_ids = input_tensor.squeeze()[answer_start:answer_stop + 1]
answer_tokens = tokenizer.convert_ids_to_tokens(answer_ids, skip_special_tokens=True)
answer = tokenizer.convert_tokens_to_string(answer_tokens)

# Print results
print(f"Question: {question}")
print(f"Answer: {answer}")

Question: 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
Answer: 1989년 2월 15일
